In [1]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
book=pd.read_csv('BX-Books.csv',sep=';', error_bad_lines=False, encoding="latin-1")
book.columns=['ISBN','booktitle','bookauthor','yearofpublication','publisher','imageurls','imageurlm','imageurll']
user=pd.read_csv('BX-Users.csv',sep=';', error_bad_lines=False,encoding="latin-1")
user.columns=['userID','Location','Age']
rating=pd.read_csv('BX-Book-Ratings.csv',sep=';', error_bad_lines=False,encoding="latin-1")
rating.columns=['userID','ISBN','bookrating']
rating.head()

b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 228957: expected 8 fields, saw 10\nSkipping line 245933: expected 8 fields, saw 9\nSkipping line 251296: expected 8 fields, saw 9\nSkipping line 259941: expected 8 fields, saw 9\nSkipping line 261529: expected 8 fields, saw 9\n'
C:\Users\SHISHIR\Anaconda3\lib\site-packages\IPython\core\interactiveshell

,userID,ISBN,bookrating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [42]:
user.head()

,userID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN


In [3]:
rating.head()

,userID,ISBN,bookrating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [4]:
book.head()

,ISBN,booktitle,bookauthor,yearofpublication,publisher,imageurls,imageurlm,imageurll
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...,http://images.amazon.com/images/P/0195153448.0...
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...,http://images.amazon.com/images/P/0002005018.0...
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...,http://images.amazon.com/images/P/0060973129.0...
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...,http://images.amazon.com/images/P/0374157065.0...
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...,http://images.amazon.com/images/P/0393045218.0...


In [10]:
combine_book_rating=pd.merge(rating,book,on='ISBN')
columns=['yearofpublication','publisher','bookauthor','imageurls','imageurlm','imageurll']
combine_book_rating=combine_book_rating.drop(columns,axis=1)
combine_book_rating.head()
print(combine_book_rating.shape)


(1031136, 4)


In [11]:
combine_book_rating=combine_book_rating.dropna(axis=0,subset=['booktitle'])
print(combine_book_rating.shape)

(1031136, 4)


In [38]:
book_ratingcount=(combine_book_rating.
                 groupby(by=['booktitle'])['bookrating'].
                 count().
                 reset_index().
                 rename(columns={'bookrating':'totalratingcount'})
                 [['booktitle','totalratingcount']]
                 )
popularity_threshold=10
book_ratingcount=book_ratingcount.query('totalratingcount>=@popularity_threshold')
book_ratingcount.describe()


,totalratingcount
count,17447.000000
mean,32.819912
std,54.367425
min,10.000000
25%,12.000000
50%,17.000000
75%,31.000000
max,2502.000000


In [19]:
rating_with_totalratingcount=combine_book_rating.merge(book_ratingcount,left_on='booktitle',right_on='booktitle',how='left')
rating_with_totalratingcount.head()
rating_with_totalratingcount['totalratingcount'].describe()

count    1.031136e+06
mean     6.978162e+01
std      1.753381e+02
min      1.000000e+00
25%      3.000000e+00
50%      1.300000e+01
75%      6.100000e+01
max      2.502000e+03
Name: totalratingcount, dtype: float64

In [14]:
print(book_ratingcount['totalratingcount'].describe())

count    241071.000000
mean          4.277312
std          16.738685
min           1.000000
25%           1.000000
50%           1.000000
75%           3.000000
max        2502.000000
Name: totalratingcount, dtype: float64


In [49]:
popularity_threshold=40
rating_popular_book=rating_with_totalratingcount.query('totalratingcount>=@popularity_threshold')
rating_popular_book.head()


,userID,ISBN,bookrating,booktitle,totalratingcount
0,276725,034545104X,0,Flesh Tones: A Novel,60
1,2313,034545104X,5,Flesh Tones: A Novel,60
2,6543,034545104X,0,Flesh Tones: A Novel,60
3,8680,034545104X,5,Flesh Tones: A Novel,60
4,10314,034545104X,9,Flesh Tones: A Novel,60


In [50]:
rating_popular_book['bookrating'].describe()

count    323258.000000
mean          3.044921
std           3.940784
min           0.000000
25%           0.000000
50%           0.000000
75%           7.000000
max          10.000000
Name: bookrating, dtype: float64

In [51]:
user_rating_pivot=rating_popular_book.pivot_table(index='booktitle',columns='userID',values='bookrating').fillna(0)
user_rating_matrix=csr_matrix(user_rating_pivot.values)

In [52]:
from sklearn.neighbors import NearestNeighbors
model_knn=NearestNeighbors(metric='cosine',algorithm='brute')
model_knn.fit(user_rating_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
         metric_params=None, n_jobs=None, n_neighbors=5, p=2, radius=1.0)

In [53]:
query_index=np.random.choice(user_rating_pivot.shape[0])
distances,indices=model_knn.kneighbors(user_rating_pivot.iloc[query_index, :].values.reshape(1,-1), n_neighbors=6)
for i in range(0, len(distances.flatten())):
    if i==0:
        print('Recommendations for {0}:\n'.format(user_rating_pivot.index[query_index]))
    else:
        print('{0}:{1},with distance of {2}:'.format(i,user_rating_pivot.index[indices.flatten()[i]],distances.flatten()[i]))
            

Recommendations for Four Past Midnight:

1:Rose Madder,with distance of 0.6732602320410359:
2:Needful Things: The Last Castle Rock Story,with distance of 0.6901501217715726:
3:The Tommyknockers,with distance of 0.6978220615129187:
4:The Dark Half,with distance of 0.7050101509237579:
5:Nightmares &amp; Dreamscapes,with distance of 0.7184163379150376:


In [2]:
user_rating_pivot2=rating_popular_book.pivot_table(index='userID',columns='booktitle',values='bookrating').fillna(0)
user_rating_pivot2.head()

NameError: name 'rating_popular_book' is not defined

In [71]:
user_rating_pivot2.shape

(47994, 2444)

In [72]:
x=user_rating_pivot2.values.T
x.shape

(2444, 47994)

In [73]:
import sklearn
from sklearn.decomposition import TruncatedSVD
SVD=TruncatedSVD(n_components=12,random_state=17)
matrix=SVD.fit_transform(x)
matrix.shape

(2444, 12)

In [76]:
import warnings
warnings.filterwarnings("ignore",category=RuntimeWarning)
corr=np.corrcoef(matrix)
corr.shape

(2444, 2444)

In [77]:
book_title=user_rating_pivot2.columns
book_list=list(book_title)
unfinished_symphony=book_list.index("Unfinished Symphony (Logan)")
print(unfinished_symphony)

2324


In [78]:
corr_unfinished_symphony=corr[unfinished_symphony]

In [83]:
list(book_title[(corr_unfinished_symphony<1.0)&(corr_unfinished_symphony>0.95)])

['All That Glitters (Landry)',
 'All Through the Night (Holiday Classics)',
 'Darkest Hour (Cutler)',
 'Full Circle',
 'Heart Song (Logan)',
 'Midnight Whispers (Cutler)',
 'Music in the Night (Logan)',
 'Now and Forever',
 'Pearl in the Mist (Landry)',
 'Ruby (Landry)',
 'Secrets of the Morning (Cutler)',
 'Silent Honor',
 'Tarnished Gold (Landry)',
 'The House on Hope Street',
 "Twilight's Child (Cutler)"]